<a href="https://colab.research.google.com/github/seihoon0111/rcs/blob/main/23_2%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 읽기

In [ ]:
import os
import pandas as pd
import numpy as np

base_src = "drive/MyDrive/recosys/data"
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,
                    sep = '|',
                    names = u_cols,
                    encoding = 'latin_1')

u_item_src = os.path.join(base_src,'u.item')

i_cols = ['movie_id','title','release date','video release data',
          'IMDB URL','unknown','Action','Adventure','Animation',
          'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep ='|',
                     names = i_cols,
                     encoding = 'latin-1')

u_data_src = os.path.join(base_src,'u.data')

r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep ='\t',
                      names = r_cols,
                      encoding='latin-1')
#drop timestamp
ratings = ratings.drop('timestamp',axis=1)
movies = movies[['movie_id','title']]

In [ ]:
from sklearn.model_selection import train_test_split

x= ratings.copy()
y= ratings['user_id']

x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size=0.25,
                                                 stratify = y)

def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

In [ ]:
def score(model):
  id_pair = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user,movie) in id_pair])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

집단 추천 model

In [ ]:
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
user_mean = x_train.groupby(['user_id'])['rating'].mean()
def bestseller(user_id,movie_id):
  try:
    rating = train_mean[movie_id]
  except:
    rating = user_mean[user_id]

  return rating

score(bestseller)

1.023711672070209

성별에 따른 사용자 집단 추천방식

In [ ]:
train_merge = pd.merge(x_train,users)
train_mean_s = train_merge.groupby(['movie_id','sex'])['rating'].mean()

def bestseller_s(user_id,movie_id):
  try:
    rating = train_mean_s[movie_id,train_merge.loc[user_id]['sex']]
  except:
    rating = user_mean[user_id]
  return rating

score(bestseller_s)

1.0302692811459468